<center style="font-size:48px;">Further Clean Up</center>

In [ ]:
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
from folium.plugins import HeatMap, MousePosition
import warnings
warnings.filterwarnings('ignore')

cars = pd.read_csv('../Data/car-assignments.csv')
cc = pd.read_csv('../Data/cc_data.csv', encoding='cp1252', parse_dates=['timestamp'])
gps = pd.read_csv('../Data/gps.csv', parse_dates=['Timestamp'])
loyalty = pd.read_csv('../Data/loyalty_data.csv', encoding='cp1252', parse_dates=['timestamp'])

In [ ]:
locations = gps.merge(cars, left_on='id', right_on='CarID', how='left')
locations.drop(columns='CarID', inplace =True)
timeUnit = ['day', 'hour', 'minute', 'second']
for unit in timeUnit:
    if unit == 'day':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.day)
    if unit == 'hour':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.hour)
    if unit == 'minute':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.minute)
    if unit == 'second':
        locations[unit] = locations['Timestamp'].apply(lambda x: x.second)
locations.head()

In [ ]:
m = folium.Map(location=[36.07, 24.87], zoom_start=14)
kronos = gpd.read_file('../Geospatial/Kronos_Island.shp')
folium.features.GeoJson(kronos,  style_function= lambda feature: {'fillColor' : 'black'}, control=False).add_to(m)
abila = gpd.read_file('../Geospatial/Abila.shp')
folium.features.GeoJson(abila, style_function= lambda feature: {'color' : 'white'}, control=False).add_to(m)

for job in locations.CurrentEmploymentType.unique():
    if job is not np.nan:
        if job == 'Facilities':
            temp = locations[(locations.CurrentEmploymentType == job) | (locations.LastName.isnull())]
            g = folium.FeatureGroup(name = job)
        else:
            temp = locations[locations.CurrentEmploymentType == job]
            g = folium.FeatureGroup(name = job, show=False)
        m.add_child(g)
        HeatMap(temp[['lat', 'long']], radius =20).add_to(g)
fmtr = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
MousePosition(position='topleft', separator=' | ', prefix="Mouse:",
              lat_formatter=fmtr, lng_formatter=fmtr).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m

In [ ]:
import numpy as np
from folium.plugins import HeatMapWithTime
# Where are cars located over the entire dataset
for job in locations.CurrentEmploymentType.unique():
    if job is not np.nan:
        m = folium.Map(location=[36.07, 24.87], zoom_start=14)
        kronos = gpd.read_file('../Geospatial/Kronos_Island.shp')
        folium.features.GeoJson(kronos,  style_function= lambda feature: {'fillColor' : 'black'}, control=False).add_to(m)
        abila = gpd.read_file('../Geospatial/Abila.shp')
        folium.features.GeoJson(abila, style_function= lambda feature: {'color' : 'white'}, control=False).add_to(m)
        weights = []
        index = []
        if job == 'Facilities':
            temp = locations[(locations.CurrentEmploymentType == job) | (locations.LastName.isnull())]
            for hour in sorted(temp.hour.unique()):
                weights.append(temp[temp.hour == hour][['lat', 'long']].groupby(['lat', 'long']).count().reset_index().values.tolist())
                index.append(pd.to_datetime(hour, format='%H').strftime('%I %p'))
                [[ x.append(0.1) for x in y] for y in weights]
        else:
            temp = locations[locations.CurrentEmploymentType == job]
            for hour in sorted(temp.hour.unique()):
                weights.append(temp[temp.hour == hour][['lat', 'long']].groupby(['lat', 'long']).count().reset_index().values.tolist())
                index.append(pd.to_datetime(hour, format='%H').strftime('%I %p'))
                [[ x.append(0.1) for x in y] for y in weights]
        HeatMapWithTime(weights, index = index).add_to(m)
        fmtr = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
        MousePosition(position='topleft', separator=' | ', prefix="Mouse:",
                    lat_formatter=fmtr, lng_formatter=fmtr).add_to(m)
        print(job)
        display(m)

In [ ]:
cc['Is_Loyalty'] = False
loyalty['Is_Loyalty'] = True
buys = pd.concat([cc, loyalty])
buys = buys.merge(cars, left_on=['LastName', 'FirstName'], right_on=['LastName', 'FirstName'])
timeUnit = ['day', 'hour', 'minute', 'second']
for unit in timeUnit:
    if unit == 'day':
        buys[unit] = buys['timestamp'].apply(lambda x: x.day)
    if unit == 'hour':
        buys[unit] = buys['timestamp'].apply(lambda x: x.hour)
    if unit == 'minute':
        buys[unit] = buys['timestamp'].apply(lambda x: x.minute)
    if unit == 'second':
        buys[unit] = buys['timestamp'].apply(lambda x: x.second)

buys

In [ ]:
# Lets see if there are any locations that are only in one dataframe
uniqueCC = set(buys[~buys.Is_Loyalty]['location']).difference(set(buys[buys.Is_Loyalty]['location']))
print('Unique locations in the credit card dataframe : {}'.format(uniqueCC))
uniqueLoyalty = set(buys[buys.Is_Loyalty]['location']).difference(set(buys[~buys.Is_Loyalty]['location']))
print('Unique locations in the loyalty dataframe : {}'.format(uniqueLoyalty))

In [ ]:
buys[buys.FirstName == 'Sten']

In [ ]:
buys[['CarID', 'location', 'day']].duplicated().sum()

In [ ]:
buys[buys[['CarID', 'location', 'day']].duplicated(keep = False)][buys.FirstName == 'Willem'].sort_values(['location', 'day'])

In [ ]:
s = buys.sort_values('Is_Loyalty')
onlyLoyalty = len(s[s[['CarID', 'location', 'day', 'price']].duplicated(keep='first')][buys.Is_Loyalty])/len(buys[buys.Is_Loyalty]) * 100
onlyCC = len(buys[buys[['CarID', 'location', 'day', 'price']].duplicated(keep = 'last')][~buys.Is_Loyalty])/len(buys[~buys.Is_Loyalty]) * 100
print('~{} % of transactions in the loyalty transactions were in the credit card transactions'.format(round(onlyLoyalty,3)) )
print('~{} % of transactions in the credit card transactions were in the loyalty transactions'.format(round(onlyCC,3)) )

In [ ]:
buys.groupby('CurrentEmploymentType')[['price']].agg([np.median, 'mean']).sort_values(('price', 'median'), ascending = False)

In [ ]:
buys.groupby('CurrentEmploymentTitle')[['price']].agg([np.median, 'mean']).sort_values(('price', 'median'), ascending = False)

In [ ]:
buys.groupby(['CurrentEmploymentType','Is_Loyalty'])[['price']].agg([np.median, 'mean'])

In [ ]:
buys.groupby(['CurrentEmploymentTitle','Is_Loyalty'])[['price']].agg([np.median, 'mean'])

<div>
    <span  style=width:700px;display:inline-block;align:left">
        <a href="./EDA.ipynb"><< Explortory Data Analysis</a>
    </span>
    <span style="width:700px;display:inline-block;" align="right">
        <a href="./CleanUp.ipynb">Data Clean Up>></a>
    </span>
</div>
<div>
    <center>
        <span style="width:250px;display:inline-block">
            <a href="../Master.ipynb">Master Notebook</a>
        </span>
        <span style="width:250px;display:inline-block">
            <a href="../README.md">Table of Contents</a>
        </span>
    <center>
</div>